In [5]:
import csv
import string
import re
import urllib.request
import threading
import time

kick_words = ["kickstarter", "campaign", "pledge", "reward", "perks", "perk", "rewards", "support", "supporting",
              "help", "backer", "backers", "update", "thank", "hours", "days", "check",
              "backed", "project","feature", "goal", "goals", "shout", "lets", "shout-out", "pledges", "product", 
              "final","minutes", "join", "featured", "halfway", "k", "stretch", "spread", "sponsor", "sponsored",
              "sponsors","order", "kickbooster","link"]

special_words = ["$", "kickstarter.com", "help us", "help me", "shout out", "days left", "hours left", "unlock",
                 "kck.st", "fund","so close", "about us"]

word_list = []
detection = []
stop_words = []
translator = str.maketrans('', '', string.punctuation)
exitFlag = 0

class Campaign():

    def __init__(self, title, cid, post, url, source, postID):
        initial_dict = {'source': source, 'text': post, 'postID': postID}
        self._title = title
        self._cid = cid
        self._post = post
        self._url = url
        self._source = source
        self._postdict = {}
        self._listofdict = []
        self._listofdict.append(initial_dict)
        self._name = '🐶'
    
    def __repr__(self):
        return repr(self._name)

    def addListOfDict(self,listofdict):
        self._listofdict = self._listofdict + listofdict

    def addListOfDict(self,listofdict):
        self._listofdict = listofdict

    def addPost(self,post,source, postID):
        tempdict = {}
        tempdict['text'] = post
        tempdict['source'] = source
        tempdict['postID'] = postID
        self._listofdict.append(tempdict)
        return self._listofdict
    

def openfileStopWords(filename):
    contents = []
    with open(filename, encoding = 'ascii', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        contents = [i.strip() for row in reader for i in row if i != '']
        csvfile.close()
    return contents

def openSmallDataFile():
    listOfCampaigns = []
    answers = []
    posts = []

    with open('PostRelatedCampaign2.csv', encoding = 'ascii', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        contents = [row for row in reader]
        csvfile.close()

    prev_obj = None

    for cell in contents[1:]:
        cid = cell[0]
        title = cell[1]
        post = cell[5]
        url = cell[3]
        source = cell[4]
        postID = cell[2]
        answers.append(cell[8])
        if prev_obj == None:
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
        elif prev_obj._cid == cid:
            prev_obj.addPost(post, source, postID)
            posts.append(postID)
        else:
            listOfCampaigns.append(prev_obj)
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
    listOfCampaigns.append(prev_obj)
    return [listOfCampaigns, answers, posts]


def title_to_list(title_text):
    global stop_words
    title = str.lower(title_text)
    title = title.translate(translator)
    words = title.split()
    the_list = [word for word in words if word not in stop_words and (word.isdigit()) == False]
    return the_list

def hashtagSorter(title,list):
    for i in list:
        if i in title.replace(" ",""):
            return True
    return False

def unshorten_url(url):
    response = urllib.request.urlopen(url)
    html = response.geturl()
    return html
    response.close()

def runlinks(shortURLS):
    while Thread._counter:
        time.sleep(delay)
        real_url_list = []
        if len(shortURLS) > 0:
            for url in shortURLS:
                try:
                    real_url = unshorten_url(url)
                    real_url_list.append(real_url)
                    #print(real_url)
                except Exception:
                    pass
    return real_url_list

    
def sorter(post_text, title_text):
    urlList = [word.partition('\\')[0] for word in post_text.split() if 'http' in word[:4] and '_'not in word]
    shortURLS = [link for link in urlList if 'https://kickstarter' not in link and 'https://www.kickstarter' not in link]
    #real_url_list = runlinks(shortURLS)
    
    post_text = str.lower(post_text)
    for s_word in special_words:
        if s_word in post_text:
            detection.append("1")
            return "1"
            
    hashtags = re.findall(r"#(\w+)", post_text)
    exist = False
    if len(hashtags) > 0:
        exist = hashtagSorter(title_text, hashtags)
    if exist == True:
        detection.append("1")
        return "1"

    post_text = str.lower(post_text).rstrip()
    post_text = post_text.translate(translator)
    for word in post_text.split():
        if word in word_list:
            detection.append("1")
            return "1"
    detection.append("0")
    return "0"



def assign(posts_list, title_text):
    global word_list
    word_list = kick_words + title_to_list(title_text)
    for post in posts_list:
        num = sorter(post['text'], title_text)
        post['kickstarter'] = num
    return posts_list


def success_rate(answer_list, complete_post_list):
    f = open('output2.csv', 'w')
    writer = csv.writer(f)
    writer.writerow( ('CategoryNames','CategoryNumbers') )
    for i,j in zip(detection, complete_post_list):
        f.write(str(i)+","+str(j)+"\n")
    f.close()

    wrong = []
    right = 0
    wrongAnswers = []
    for det, ans, post in zip(detection, answer_list, complete_post_list):
        if det == ans:
            right += 1
        else:
            wrong.append(post)
            wrongAnswers.append(det)
    return [(right/len(detection) *100), wrong, wrongAnswers]

def main():
    global stop_words
    output = openSmallDataFile() #Joe's code
    stop_words = openfileStopWords('stop-word-list.csv')

    stop_words.append("best")
    stop_words.append("short")
    stop_words.append("big")
    stop_words.append("every")

    campaign_list = output[0]
    answer_list = output[1]
    complete_post_list = output[2]
    new_campaign_list = []

    for campaign in campaign_list:
        postsList = campaign._listofdict
        newPostsList = assign(postsList, campaign._title)
        campaign.posts = newPostsList
        new_campaign_list.append(campaign)

    output2 = success_rate(answer_list, complete_post_list)
    #print(output2[0])


if __name__ == '__main__':
    main()

In [93]:
def openLargeDataFile():
    """[0:'', '1:id', 2:'handle', 3:'postDate', 4:'postType', 5:'postUrl', 6:'source', 
    7:'message', 8:'comments', 9:'shares', 10:'likes', 11:'views', 12:'CampaignId', 13:'campaign_name', 
    14:'campaign_startDate', 15:'campaign_endDate', 16:'project_goal', 17:'project_currency', 18:'project_staffPick', 
    19:'project_category', 20:'project_url', 21:'project_pledgeAmount', 21:'project_backerCount', 22:'intents',
    23:'Succeeded']"""

    Large_listOfCampaigns = []
    Large_answers = []
    Large_posts = []

    with open('LargeData.csv', encoding = 'ascii', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        contents = [row for row in reader]
        csvfile.close()
    tables = contents[0]
    print(tables)

    prev_obj = None

    for cell in contents[1:]:
        cid = cell[12]
        title = cell[13]
        post = cell[7]
        url = cell[20]
        source = cell[6]
        postID = cell[1]
        Large_answers.append(cell[8])
        if prev_obj == None:
            obj = Campaign(title, cid, post, url, source, postID)
            Large_posts.append(postID)
            prev_obj = obj
        elif prev_obj._cid == cid:
            prev_obj.addPost(post, source, postID)
            Large_posts.append(postID)
        else:
            Large_listOfCampaigns.append(prev_obj)
            obj = Campaign(title, cid, post, url, source, postID)
            Large_posts.append(postID)
            prev_obj = obj

    return [listOfCampaigns, answers, posts]
